# Using the Babel-wrapped Meteorology component

In this example, I want to give the `Meteorology` component a constant scalar precipitation value and see whether it produces output when the model state is updated.

Start with an import and some magic:

In [ ]:
%matplotlib inline
import numpy as np

Import the `Meteorology` component and create an instance:

In [ ]:
from cmt.components import Meteorology

In [ ]:
m = Meteorology()

Locate the **cfg** file:

In [ ]:
cfg_file = './input/meteorology.cfg'

Initialize the component with the **cfg** file:

In [ ]:
m.initialize(cfg_file)

Note that I set `P` (the precipitation rate) to 20.0 mm/hr in the **cfg** file. Why does Meteorology now report a different value?  
Aha! On line 552 of **met_base.py**, the constant `mmph_to_mps` is defined:

In [ ]:
mmph_to_mps = (np.float64(1) / np.float64(3600000))

And on line 764 of **met_base.py** it's applied the input value of `P`:

In [ ]:
print 20.0 * mmph_to_mps

So it looks like the `[mmph]` label in the `initialize` output should be `[m/s]`. A typo.

However, if I call `get_value` at this point, the model precip values are still zeroed out:

In [ ]:
precip = m.get_value('atmosphere_water__precipitation_leq-volume_flux')  # `P` internally
print precip.shape
print precip.min(), precip.max(), precip.mean()

 Maybe this will change after the first update?

Show the model start, current, and stop times:

In [ ]:
print m.get_start_time(), m.get_current_time(), m.get_end_time()

Get the model time step:

In [ ]:
time_step = m.get_value('model__time_step')  # `dt` internally
time_step = time_step.max()
print time_step

Advance the model by one time step:

In [ ]:
m.update(time_step)

In [ ]:
print m.get_start_time(), m.get_current_time(), m.get_end_time()

Note that it hasn't precipitated:

In [ ]:
precip = m.get_value('atmosphere_water__precipitation_leq-volume_flux')
print precip.min(), precip.max(), precip.mean()

But this might be expected, since the initial value retrieved by `get_value` was also zero.

Try manually setting the precip rate variable to a non-zero value:

In [ ]:
m.set_value('atmosphere_water__precipitation_leq-volume_flux', 15.0)
precip = m.get_value('atmosphere_water__precipitation_leq-volume_flux')
print precip.shape
print precip.min(), precip.max(), precip.mean()

Advance the model by another time step:

In [ ]:
m.update(2*time_step)